In [2]:
# Question 1
import pandas as pd

df = pd.read_csv('Project1-Classification.csv')
df.head(10)

,full_text,summary,keywords,publish_date,authors,url,leaf_label,root_label
0,"'Pure Hockey, the largest hockey retailer in t...","'Pure Hockey, the largest hockey retailer in t...","['acquire', 'agrees', 'nevada', 'retail', 'hoc...",2022-10-21 17:11:22+00:00,['Ein News'],https://www.einnews.com/pr_news/597148062/pure...,hockey,sports
1,'HKO Hockey Canada Sponsorship 20221006\n\nHoc...,'The tally of Hockey Canada’s sponsorship loss...,"['sexual', 'exclusive', 'settlement', 'million...",NaN,['Barbara Shecter'],https://ca.finance.yahoo.com/news/exclusive-ho...,hockey,sports
2,'Canadian Prime Minister Justin Trudeau said W...,'Canadian Prime Minister Justin Trudeau said W...,"['québec', 'surprise', 'trust', 'sexual', 'org...",2022-10-05 00:00:00,['The Athletic Staff'],https://theathletic.com/3659888/2022/10/05/hoc...,hockey,sports
3,'Hockey Canada paid a crisis communications fi...,'Nicholson says that Hockey Canada did not go ...,"['sexual', 'organization', 'million', 'ceo', '...",2022-11-15 00:00:00,['Dan Robson'],https://theathletic.com/3900630/2022/11/15/hoc...,hockey,sports
4,'The Denver Broncos are looking for a new head...,'The Denver Broncos are looking for a new head...,"['beats', 'force', 'season', 'teams', 'anchora...",NaN,"['Parker Seibold', 'The Gazette', 'Gazette File']",https://gazette.com/multimedia/photo-air-force...,hockey,sports
5,'Referee Martin Pronovost wears a Hockey Canad...,'Hockey Canada reached a settlement in May wit...,"['situation', 'culture', 'needs', 'sexual', 'i...",NaN,['Sara Civian'],https://bleacherreport.com/articles/10052673-m...,hockey,sports
6,'Sonny Milano tied it with less than three min...,'Flyers Capitals Hockey Washington Capitals le...,"['ovechkins', 'ot', 'goal', 'game', 'philadelp...",2022-11-24 05:26:00+00:00,['News Traffic Weather'],https://wtop.com/washington-capitals/2022/11/a...,hockey,sports
7,'GRAND FORKS — A hockey stick in pieces used t...,'GRAND FORKS — A hockey stick in pieces used t...,"['stick', 'repairs', 'process', 'dakota', 'nor...",2022-11-29 13:33:00,"['A South Dakota Native', 'Hunter Joined Forum...",https://www.inforum.com/business/a-broken-hock...,hockey,sports
8,'Alex Ovechkin tied Gordie Howe for the most g...,'Capitals Red Wings Hockey Detroit Red Wings c...,"['ovechkin', 'wings', 'red', 'howes', 'osorio'...",2022-11-04 02:51:00+00:00,['News Traffic Weather'],https://wtop.com/washington-capitals/2022/11/o...,hockey,sports
9,'Hockey Canada’s members have elected a new bo...,'Hockey Canada’s members have elected a new bo...,"['elect', 'sexual', 'organization', 'allegatio...",2022-12-17 00:00:00,['The Athletic Staff'],https://theathletic.com/4011708/2022/12/17/hoc...,hockey,sports
